In [63]:
import pandas as pd
import numpy as np


In [87]:
## Load datasets

rosters=pd.read_csv("PKL_Rosters_Seasons_1-10.csv")
matches=pd.read_csv("PKL_All_Seasons_Matches.csv")

In [88]:
rosters.head(3)

,Player ID,Name,Jersey Number,Captain Count,Played Count,Green Card Count,Yellow Card Count,Red Card Count,Starter Count,Top Raider Count,Top Defender Count,Total Points,Team ID,Team Name,Total Matches in Season,season,team_name
0,15,Sanjay Shrestha,88,0,2,0,0,0,1,0,0,0,3,Jaipur Pink Panthers,16,1,Jaipur Pink Panthers
1,22,Wei Yang,20,0,4,0,0,0,1,0,0,2,3,Jaipur Pink Panthers,16,1,Jaipur Pink Panthers
2,32,Balbir Singh,1,0,8,0,0,0,2,0,0,14,3,Jaipur Pink Panthers,16,1,Jaipur Pink Panthers


In [89]:
matches.head(3)

,Season,Match_ID,Match_Name,League_Stage,Year,Venue,Match_Outcome,Start_Date,End_Date,Result,Winning Margin,team_score_1,team_score_2,team_name_1,team_id_1,team_name_2,team_id_2
0,1,1,Match 1,League,2014,"Dome,Nsci,Svp Stadium,Mumbai",U Mumba beat Jaipur Pink Panthers (45 - 31),07/26/2014T8:00:00TPM,07/26/2014T8:00:00TPM,NaN,14.0,45,31,U Mumba,5,Jaipur Pink Panthers,3
1,1,2,Match 2,League,2014,"Dome,Nsci,Svp Stadium,Mumbai",Bengaluru Bulls beat Dabang Delhi (47 - 28),07/26/2014T9:00:00TPM,07/26/2014T9:00:00TPM,NaN,19.0,28,47,Dabang Delhi,2,Bengaluru Bulls,1
2,1,3,Match 3,League,2014,"Dome,Nsci,Svp Stadium,Mumbai",Bengaluru Bulls beat Puneri Paltan (40 - 37),07/27/2014T8:00:00TPM,07/27/2014T8:00:00TPM,NaN,3.0,40,37,Bengaluru Bulls,1,Puneri Paltan,7


In [90]:
print("Matches shape:", matches.shape)
print("Rosters shape:", rosters.shape)

Matches shape: (1064, 17)
Rosters shape: (1962, 17)


### Cleaning Match dataset.

In [91]:
# Step 1) Standardize column names

m = matches.copy()
m.columns = m.columns.str.strip().str.lower().str.replace(' ', '_', regex=False)

print("After renaming columns:")
print(m.columns.tolist()[:15])
m.head(2)


After renaming columns:
['season', 'match_id', 'match_name', 'league_stage', 'year', 'venue', 'match_outcome', 'start_date', 'end_date', 'result', 'winning_margin', 'team_score_1', 'team_score_2', 'team_name_1', 'team_id_1']


,season,match_id,match_name,league_stage,year,venue,match_outcome,start_date,end_date,result,winning_margin,team_score_1,team_score_2,team_name_1,team_id_1,team_name_2,team_id_2
0,1,1,Match 1,League,2014,"Dome,Nsci,Svp Stadium,Mumbai",U Mumba beat Jaipur Pink Panthers (45 - 31),07/26/2014T8:00:00TPM,07/26/2014T8:00:00TPM,NaN,14.0,45,31,U Mumba,5,Jaipur Pink Panthers,3
1,1,2,Match 2,League,2014,"Dome,Nsci,Svp Stadium,Mumbai",Bengaluru Bulls beat Dabang Delhi (47 - 28),07/26/2014T9:00:00TPM,07/26/2014T9:00:00TPM,NaN,19.0,28,47,Dabang Delhi,2,Bengaluru Bulls,1


In [92]:
####### Step 2)  Drop start and end date columns
m.drop(columns=['start_date','end_date'],inplace=True)


In [93]:
######## Step 3)  Cleaning winning_margin column  and creating total points column.
##   It is noticed that winning_margin is null where there is a tie. so we fill such instances of wining_margin with 0

def fill_winning_margin(row):
    if row['team_score_1']>row['team_score_2']:
        return row['team_score_1']-row['team_score_2']
    elif row['team_score_1']<row['team_score_2']:
        return row['team_score_2']-row['team_score_1']
    else:
        return 0

m['winning_margin']=m.apply(fill_winning_margin,axis=1)
m['total_points_match'] = m['team_score_1'] + m['team_score_2']

In [94]:
########## Step 4)  Drop Unnecessary columns ['match_outcome','result'] and maintain culumns named as winner and loser.

m2=m.drop(columns=['match_outcome','result'],errors='ignore')

def get_winner(row):
    if row['team_score_1']>row['team_score_2']:
        return row['team_name_1']
    elif row['team_score_1']<row['team_score_2']:
        return row['team_name_2']
    else:
        return 'tied'

def get_loser(row):
    if row['team_score_1']>row['team_score_2']:
        return row['team_name_2']
    elif row['team_score_1']<row['team_score_2']:
        return row['team_name_1']
    else:
        return 'tied'
        
m2['winner']=m2.apply(get_winner,axis=1)
m2['loser']=m2.apply(get_loser,axis=1)


###########  Step 5) Replace Null values in venue with 'unknown venue'
m2['venue']= m2['venue'].fillna('unknown_venue')

########## Step 6)  Max score of match
m2['max_score']=m2[['team_score_1','team_score_2']].max(axis=1)

m2.head(3)

,season,match_id,match_name,league_stage,year,venue,winning_margin,team_score_1,team_score_2,team_name_1,team_id_1,team_name_2,team_id_2,total_points_match,winner,loser,max_score
0,1,1,Match 1,League,2014,"Dome,Nsci,Svp Stadium,Mumbai",14,45,31,U Mumba,5,Jaipur Pink Panthers,3,76,U Mumba,Jaipur Pink Panthers,45
1,1,2,Match 2,League,2014,"Dome,Nsci,Svp Stadium,Mumbai",19,28,47,Dabang Delhi,2,Bengaluru Bulls,1,75,Bengaluru Bulls,Dabang Delhi,47
2,1,3,Match 3,League,2014,"Dome,Nsci,Svp Stadium,Mumbai",3,40,37,Bengaluru Bulls,1,Puneri Paltan,7,77,Bengaluru Bulls,Puneri Paltan,40


###  Cleaning Rosters dataset

In [95]:
# Step 1)  Clean rosters:  Standardize column names

r = rosters.copy()
r.columns = r.columns.str.strip().str.lower().str.replace(' ', '_', regex=False)

r.head()

,player_id,name,jersey_number,captain_count,played_count,green_card_count,yellow_card_count,red_card_count,starter_count,top_raider_count,top_defender_count,total_points,team_id,team_name,total_matches_in_season,season,team_name
0,15,Sanjay Shrestha,88,0,2,0,0,0,1,0,0,0,3,Jaipur Pink Panthers,16,1,Jaipur Pink Panthers
1,22,Wei Yang,20,0,4,0,0,0,1,0,0,2,3,Jaipur Pink Panthers,16,1,Jaipur Pink Panthers
2,32,Balbir Singh,1,0,8,0,0,0,2,0,0,14,3,Jaipur Pink Panthers,16,1,Jaipur Pink Panthers
3,44,Gangadhari Mallesh,0,0,10,0,0,0,3,0,0,15,3,Jaipur Pink Panthers,16,1,Jaipur Pink Panthers
4,52,Jasvir Singh,2,0,15,1,0,0,15,0,0,148,3,Jaipur Pink Panthers,16,1,Jaipur Pink Panthers


In [14]:
#  Cards total
for c in ['green_card_count','yellow_card_count','red_card_count']:
    if c not in r.columns:
        r[c] = 0

r['cards_total'] = r['green_card_count'] + r['yellow_card_count'] + r['red_card_count']

print(r[['name','green_card_count','yellow_card_count','red_card_count','cards_total']].head())


                 name  green_card_count  yellow_card_count  red_card_count  \
0     Sanjay Shrestha                 0                  0               0   
1            Wei Yang                 0                  0               0   
2        Balbir Singh                 0                  0               0   
3  Gangadhari Mallesh                 0                  0               0   
4        Jasvir Singh                 1                  0               0   

   cards_total  
0            0  
1            0  
2            0  
3            0  
4            1  


In [96]:
#  Final checks
print(" Clean matches columns:", len(m.columns))
print(" Clean rosters columns:", len(r.columns))

m2.info()
r.info()


 Clean matches columns: 16
 Clean rosters columns: 17
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1064 entries, 0 to 1063
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   season              1064 non-null   int64 
 1   match_id            1064 non-null   int64 
 2   match_name          1064 non-null   object
 3   league_stage        1064 non-null   object
 4   year                1064 non-null   int64 
 5   venue               1064 non-null   object
 6   winning_margin      1064 non-null   int64 
 7   team_score_1        1064 non-null   int64 
 8   team_score_2        1064 non-null   int64 
 9   team_name_1         1064 non-null   object
 10  team_id_1           1064 non-null   int64 
 11  team_name_2         1064 non-null   object
 12  team_id_2           1064 non-null   int64 
 13  total_points_match  1064 non-null   int64 
 14  winner              1064 non-null   object
 15  loser             

In [97]:
#  Optional — Save cleaned versions

m2.to_csv("pkl_matches_cleaned.csv", index=False)
r.to_csv("pkl_rosters_cleaned.csv", index=False)

print("Cleaned files saved successfully.")

Cleaned files saved successfully.
